# Stocks
This notebook shows how to treat the stock prices and make some simple analyses of financial time series.

## How to read and handle a csv file
Here we open a csv (comma separated values) file containing the stock prices (in this case, daily close prices).

In [ ]:
import pandas as pd
filename = '' # add here the name of your file
prices = pd.read_csv(filename, index_col=0, parse_dates=[0])
# index_col=0 to define the first column as the index; parse_dates=[0] to read the first column as a DateTime object
prices.index = pd.to_datetime(prices.index) # to be sure that the index is in the pandas DateTime format
print(prices)

## Plot stock daily prices between a certain time range
Define the time range and the stock we want to show, and plot the corresponding close prices.

In [ ]:
stock = 'AAPL'
start_month = '2021-01'
end_month = '2021-05'
prices_toPlot = prices[[stock]][start_month:end_month]
prices_toPlot.plot()
pl.xlabel('Time', fontsize=14)
pl.ylabel('Price (USD)', fontsize=14)
pl.title(stock + ', close price', fontsize=14)

## Daily returns
Calculate daily returns and their statistics. Daily returns are percentage changes between consecutive prices. For example, if yesterday's price was 150$ and today's price is 152$, then the daily return is (152-150)/150 = 0.013, i.e. 1.3%

In [ ]:
rets = prices.pct_change()
print('The mean daily returns are:')
print(rets.mean())
print('The standard deviation of the daily returns are:')
print(rets.std())
# standard deviations of daily returns show the level of fluctuations of the stocks;
# higher standard deviation implies higher stock volatility (see "Calculate stock volatility")

## Filtering and grouping
We can filter the returns dataframe based on some condition (for example, the magnitude of the returns).

In [ ]:
return_threshold = 2 # we will select only the days with daily return larger than 2%
stock = 'AAPL'
rets_percent = 100*rets #  transform returns into percent returns to increase readability
rets_percent_large = rets_percent[[stock]][rets_percent > return_threshold] # in the square brackets, we set the condition
rets_percent_small = rets_percent[[stock]][rets_percent < -return_threshold]
rets_percent_large

## Calculate stock volatility
From the stock prices and returns we can calculate the volatility, i.e. the average fluctuations of the stock in the last n days. The volatility represents the risk associated to the financial object: the higher the volatility, the riskier the security. In mathematical terms, it is nothing but the weighted variance of the stock returns multiplied by a factor, as shown in the function below.

In [ ]:
import numpy
import datetime

def volaCalc2(x, w): # x is your pandas dataframe showing the close-close returns and w is a vector of weights
    # in this case, we calculate the volatility on 65 trading days, and we weight the last 5 days twice w.r.t. to the others:
    w = np.append(np.repeat(1, 60),np.repeat(2, 5)) 
    ttt=datetime.datetime.now() # check time
    
    n_col = x.shape[1]
    n_row = x.shape[0]
    n_w = len(w)
    s_w = sum(w)
    tmp_mean = np.tile(np.nan, (n_row, n_col))
    vola = np.tile(np.nan, (n_row, n_col))
    x_array = x.values # transform pandas into array
    
    for j in range(0,n_col): # loop on columns
        tmpcol=x_array[:,j];
        for i in range(0,n_row): # loop on rows
            if i >= n_w:
                tmp_mean[i,j] = sum((w*tmpcol[i+1-n_w:i+1]))/s_w
                vola[i,j] = sum(w*(tmpcol[i+1-n_w:i+1]-tmp_mean[i,j])*(tmpcol[i+1-n_w:i+1]-tmp_mean[i,j]))/(s_w-1) #weighted variance
                
    vola = pd.DataFrame(vola, index = x.index, columns=x.columns)
    print(' total time: '+str((datetime.datetime.now() - ttt).seconds + (datetime.datetime.now() - ttt).microseconds/1E6) + ' s')        
    return(vola)

weights = np.append(np.repeat(1, 60),np.repeat(2, 5))
volatility_daily = np.sqrt(volaCalc2(rets, weights)) # compute the square root of the variance
# this is the daily volatility; usually, though, the volatility is presented in annualized terms 
# assuming there are 252 trading days in a year, we therefore multiply the daily volatility by the square root of 252:
volatility = np.sqrt(252)*volatility